In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, power_transform
from sklearn.model_selection import train_test_split
from tensorflow import keras

KeyboardInterrupt: 

In [3]:
# Load the dataset
#from google.colab import drive
#drive.mount('/content/drive')
#df = pd.read_csv('/content/drive/My Drive/dataset/csgo_round_snapshots.csv')


#iNPUT csgo_round_snapshots.csv
from google.colab import files
df = files.upload() #uploadไฟล์

# Split X and y
y = df['round_winner']
X = df.drop(['round_winner'], axis=1)
# Drop grenade info columns
X = X.drop(X.columns[X.columns.str.contains('grenade')], axis=1)

KeyboardInterrupt: 

In [ ]:
# เลือกเฉพาะ 9 ฟีเจอร์ที่ต้องการใช้ในการเทรน
selected_features = ['time_left', 'ct_score', 't_score', 'bomb_planted',
                     'ct_health', 't_health', 'ct_armor', 't_armor']
X = X[selected_features]

In [ ]:
# Encode categorical columns
object_cols = [ 'bomb_planted']
ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
X_encoded = pd.DataFrame(ohe.fit_transform(X[object_cols]))
X_encoded.columns = ohe.get_feature_names_out(object_cols)
X_encoded.index = X.index

In [ ]:
# Combine with numerical features
X = pd.concat([X.drop(object_cols, axis=1), X_encoded], axis=1)

# Encode target variable
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)  # "CT" → 0, "T" → 1

# Normalize numerical features
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

In [ ]:
# Train-test split
X_train_full, X_test, y_train_full, y_test = train_test_split(
    X_scaled, y, stratify=y, test_size=0.1, random_state=0)

X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, stratify=y_train_full, test_size=0.25, random_state=0)

In [ ]:
print(X.columns.tolist())

In [ ]:
# Build Neural Network model
model = keras.models.Sequential()
model.add(keras.layers.BatchNormalization(input_shape=[X_train.shape[1]]))

for _ in range(4):
    model.add(keras.layers.Dense(300, kernel_initializer="he_normal", use_bias=False))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Activation("elu"))
    model.add(keras.layers.Dropout(rate=0.2))

In [ ]:
# Output layer (softmax for multi-class classification)
model.add(keras.layers.Dense(2, activation="softmax"))  # 2 classes: CT, T

In [ ]:
# Compile model
model.compile(
    loss='sparse_categorical_crossentropy',  # Categorical loss function
    optimizer='Nadam',
    metrics=['accuracy', keras.metrics.AUC(), keras.metrics.Precision(), keras.metrics.Recall()]
)

In [ ]:
# Callbacks
reduce_lr_cb = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow import keras

# Make a train, validation and test set
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y,
 stratify=y, test_size=0.1, random_state=0)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full,
 stratify=y_train_full, test_size=0.25, random_state=0)

# Set model parameters
n_layers = 4
n_nodes = 300
regularized = False
dropout = True
epochs = 50

# Make a Keras DNN model
model = keras.models.Sequential()
model.add(keras.layers.BatchNormalization())
for n in range(n_layers):
    if regularized:
        model.add(keras.layers.Dense(n_nodes, kernel_initializer="he_normal",
         kernel_regularizer=keras.regularizers.l1(0.01), use_bias=False))
    else:
        model.add(keras.layers.Dense(n_nodes,
         kernel_initializer="he_normal", use_bias=False))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Activation("elu"))
    if dropout:
        model.add(keras.layers.Dropout(rate=0.2))
model.add(keras.layers.Dense(1, activation="sigmoid"))
model.compile(loss='binary_crossentropy', optimizer='Nadam', metrics=['accuracy'])

# Make a callback that reduces LR on plateau
reduce_lr_cb = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                                                 patience=5, min_lr=0.001)

# Make a callback for early stopping
early_stopping_cb = keras.callbacks.EarlyStopping(patience=5)

# Train DNN.
history = model.fit(np.array(X_train), np.array(y_train), epochs=epochs,
     validation_data=(np.array(X_valid), np.array(y_valid)),
      callbacks=[reduce_lr_cb, early_stopping_cb], batch_size=128)

In [ ]:
# Evaluate the test set
model.evaluate(X_test, y_test)

In [ ]:
# บันทึกโมเดลเป็นไฟล์ .h5
#model.save("dnn_model.keras")

In [ ]:
# prompt: Make use of a model to test the model is predict correcly

import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
# ... (your existing code) ...

# Evaluate the model on the test set
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int) # Convert probabilities to class labels

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy}")

# Generate classification report
print(classification_report(y_test, y_pred))

# Generate confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

# Example: predict on a single data point
# Assuming X_test is a pandas DataFrame
example_index = 0  # Replace with the index of the example you want to test
example_data = X_test.iloc[[example_index]] # Select a single row
predicted_prob = model.predict(example_data)
predicted_class = (predicted_prob > 0.5).astype(int)[0][0]  # Get the predicted class

print("\nExample prediction:")
print(f"Actual class: {y_test[example_index]}")
print(f"Predicted probability: {predicted_prob[0][0]}")
print(f"Predicted class: {predicted_class}")


In [ ]:
# prompt: Save me a model file .h5 for DNN  to show in my streamlit

# Save the model to a .h5 file
model.save("dnn_model.h5")

# Download the .h5 file to your local machine
from google.colab import files
files.download("dnn_model.h5")
